In [58]:
from __future__ import division
import pandas as pd
import pickle
import numpy as np

pd.options.mode.chained_assignment = None

In [59]:
df = pd.read_csv("~/donorschoose/data_files/opendata_projects.csv", parse_dates = ['date_posted', 'date_completed', 'date_thank_you_packet_mailed', 'date_expiration'])
print "Done reading csv"

Done reading csv


In [60]:
"""# Pickle dump currently live projects as hold out set for final testing
live_df = df[df['funding_status'] =='live']
with open('holdout_set.pkl', 'w') as live_file:
    pickle.dump(live_df, live_file)
print 'Done creating holdout set'"""

"# Pickle dump currently live projects as hold out set for final testing\nlive_df = df[df['funding_status'] =='live']\nwith open('holdout_set.pkl', 'w') as live_file:\n    pickle.dump(live_df, live_file)\nprint 'Done creating holdout set'"

In [61]:
# Throw out reallocated and live projects and label rows
df2 = df[df['funding_status'] != "reallocated"]
df2 = df2[df2['funding_status'] != "live"]
df2['RESP'] = 0
df2['RESP'][df2['funding_status'] == 'completed'] = 1
per_remaining = len(df2) / len(df)
print "Percent of original data remaining: %0.2f" % (per_remaining * 100)
print "Done step 1/5"

Percent of original data remaining: 96.28
Done step 1/5


In [62]:
# Replace binary features with 0,1
binary_features = ['school_charter',
 'school_magnet',
 'school_year_round',
 'school_nlns',
 'school_kipp',
 'school_charter_ready_promise',
'teacher_teach_for_america',
 'teacher_ny_teaching_fellow']
for feature in binary_features:
    df2[feature] = df2[feature].replace("t", 0)
    df2[feature] = df2[feature].replace("f", 1)
print "Done step 2/5"

Done step 2/5


In [63]:
# Calculate month, quarter and year for date project was posted on site
df2['month'] = df2.date_posted.map(lambda x: x.month)
df2['quarter'] = df2.date_posted.map(lambda x: x.quarter)
df2['year'] = df2.date_posted.map(lambda x: x.year)
print "Done step 3/5"

Done step 3/5


In [64]:
# Feature engineering (expiration dates and amount/percentage of optional support)
df2['time_to_expire'] = df2.date_expiration - df2.date_posted
df2['amount_optional_support'] = df2.total_price_including_optional_support - df2.total_price_excluding_optional_support
df2['optional_support'] = 0
df2['optional_support'][df2['amount_optional_support'] > 0] = 1
df2['per_optional_support'] = df2.amount_optional_support / df2.total_price_excluding_optional_support
print "Done step 4/5"

Done step 4/5


In [65]:
# Feature engineering (previous projects posted by school & teacher)
school_posted = df2.set_index('date_posted').groupby('_schoolid').cumcount()
df2['school_previous_projects'] = school_posted.values

teacher_posted = df2.set_index('date_posted').groupby('_teacher_acctid').cumcount()
df2['teacher_previous_projects'] = teacher_posted.values
print "Done step 5/5"

Done step 5/5


In [66]:
df2.students_reached = df2.students_reached +1

In [67]:
df2.total_price_excluding_optional_support = df2.total_price_excluding_optional_support + 1

In [68]:
df2.total_price_including_optional_support = df2.total_price_including_optional_support + 1

In [69]:
df2.vendor_shipping_charges = df2.vendor_shipping_charges + 1

In [70]:
df2['log_price_including'] = np.log(df2.total_price_including_optional_support)

In [71]:
df2['log_price_excluding'] = np.log(df2.total_price_excluding_optional_support)

In [72]:
df2['sqrt_students_reached'] = np.sqrt(df2.students_reached)

In [73]:
df2['log_vendor_shipping'] = np.log(df2.vendor_shipping_charges)

In [78]:
df2['student_bins'] = pd.qcut(df2['students_reached'], 10, labels = False)

In [80]:
df2['price_per_student'] = df2.total_price_including_optional_support/df2.students_reached

In [81]:
df2['price_in_bins'] = pd.qcut(df2['total_price_including_optional_support'], 10, labels = False)
df2['price_ex_bins'] = pd.qcut(df2['total_price_excluding_optional_support'], 10, labels = False)
df2['price_per_student_bins'] = pd.qcut(df2['price_per_student'], 10, labels = False)

In [82]:
df2['teacher_gender'] = df2.teacher_prefix
df2.teacher_gender = df2.teacher_gender.replace("Mrs.", "Female")
df2.teacher_gender = df2.teacher_gender.replace("Ms.", "Female")
df2.teacher_gender = df2.teacher_gender.replace("Mr.", "Male")
df2.teacher_gender = df2.teacher_gender.replace("Dr.", np.nan)
df2.teacher_gender = df2.teacher_gender.replace("Mr. & Mrs.", np.nan)

In [83]:
df2.school_state = df2.school_state.replace("La", "LA")

In [84]:
state_df = pd.DataFrame(df2.num_donors.groupby([df2.school_state]).sum())
state_df['school_state'] = state_df.index

In [85]:
state_projects_df = pd.DataFrame(df2.num_donors.groupby([df2.school_state]).count())
state_projects_df['school_state'] = state_projects_df.index
state_df = state_df.merge(state_projects_df, on = "school_state", how = "left")
state_df.columns = ['total_state_donors', 'school_state', 'total_state_projects']

In [86]:
df2 = df2.merge(state_df, on = "school_state", how = "left")

In [87]:
df2['state_avg_donors'] = df2.total_state_donors/df2.total_state_projects

In [88]:
df2.vendor_shipping_charges = df2.vendor_shipping_charges.replace(np.nan, df2.vendor_shipping_charges.mean())

In [27]:
df3 = df2[['_projectid',
 '_teacher_acctid',
 '_schoolid',
 'school_state',
 'school_metro',
 'school_charter',
 'school_magnet',
 'school_year_round',
 'school_nlns',
 'school_kipp',
 'school_charter_ready_promise',
 'teacher_teach_for_america',
 'teacher_ny_teaching_fellow',
 'primary_focus_subject',
 'primary_focus_area',
 'resource_type',
 'poverty_level',
 'grade_level',
 'vendor_shipping_charges',
 'total_price_excluding_optional_support',
 'total_price_including_optional_support',
 'students_reached',
 'date_posted',
 'RESP',
 'month',
 'quarter',
 'year',
 'time_to_expire',
 'optional_support',
 'school_previous_projects',
 'teacher_previous_projects',
 'log_price_including',
 'log_price_excluding',
 'sqrt_students_reached',
 'student_bins',
 'price_in_bins',
 'price_ex_bins',
 'teacher_gender',
 'price_per_student',
 'price_per_student_bins',
 'total_state_donors',
 'total_state_projects',
 'state_avg_donors']]

In [28]:
df3 = df3.dropna(subset = ['school_metro','grade_level', 'students_reached', 'primary_focus_area', 'primary_focus_subject', 'resource_type', 'time_to_expire', 'teacher_gender'])

In [29]:
"""features = df3.columns.values.tolist()
for feature in features:
    per = (len(df3[feature]) - df3[feature].count())/len(df3[feature])
    if per > 0.0:
        print feature, "has %0.4f percent missing values" % (per*100)"""

'features = df3.columns.values.tolist()\nfor feature in features:\n    per = (len(df3[feature]) - df3[feature].count())/len(df3[feature])\n    if per > 0.0:\n        print feature, "has %0.4f percent missing values" % (per*100)'

In [30]:
final_remaining = len(df3)/len(df2)
print "Percent of remaining data still remaining: %0.2f" % (final_remaining * 100)

Percent of remaining data still remaining: 88.72


In [35]:
features = df3.columns.values.tolist()
features

['_projectid',
 '_teacher_acctid',
 '_schoolid',
 'school_state',
 'school_metro',
 'school_charter',
 'school_magnet',
 'school_year_round',
 'school_nlns',
 'school_kipp',
 'school_charter_ready_promise',
 'teacher_teach_for_america',
 'teacher_ny_teaching_fellow',
 'primary_focus_subject',
 'primary_focus_area',
 'resource_type',
 'poverty_level',
 'grade_level',
 'vendor_shipping_charges',
 'total_price_excluding_optional_support',
 'total_price_including_optional_support',
 'students_reached',
 'date_posted',
 'RESP',
 'month',
 'quarter',
 'year',
 'time_to_expire',
 'optional_support',
 'school_previous_projects',
 'teacher_previous_projects',
 'log_price_including',
 'log_price_excluding',
 'sqrt_students_reached',
 'student_bins',
 'price_in_bins',
 'price_ex_bins',
 'teacher_gender',
 'price_per_student',
 'price_per_student_bins',
 'total_state_donors',
 'total_state_projects',
 'state_avg_donors']

In [51]:
data_type_dict = {'_projectid': 'regular',
 '_teacher_acctid': 'regular',
 '_schoolid': 'regular',
 'school_state': 'regular',
 'school_metro': 'nominal',
 'school_charter':'regular',
 'school_magnet':'regular',
 'school_year_round':'regular',
 'school_nlns':'regular',
 'school_kipp':'regular',
 'school_charter_ready_promise':'regular',
 'teacher_teach_for_america':'regular',
 'teacher_ny_teaching_fellow':'regular',
 'primary_focus_subject': 'nominal',
 'primary_focus_area': 'nominal',
 'resource_type': 'nominal',
 'poverty_level': 'nominal',
 'grade_level': 'nominal',
 'vendor_shipping_charges':'regular',
 'total_price_excluding_optional_support':'regular',
 'total_price_including_optional_support':'regular',
 'students_reached':'regular',
 'date_posted':'regular',
 'month':'regular',
 'quarter':'regular',
 'year':'regular',
 'time_to_expire':'regular',
 'optional_support':'regular',
 'school_previous_projects':'regular',
 'teacher_previous_projects':'regular',
 'log_price_including':'regular',
 'log_price_excluding':'regular',
 'sqrt_students_reached':'regular',
 'student_bins': 'nominal',
 'price_in_bins': 'nominal',
 'price_ex_bins': 'nominal',
 'teacher_gender': 'nominal',
 'price_per_student':'regular',
 'price_per_student_bins': 'nominal',
 'total_state_donors':'regular',
 'total_state_projects':'regular',
 'state_avg_donors':'regular',
 'RESP':'regular'}

In [52]:
def dummy_variables(data, data_type_dict):
    #Loop over nominal variables.
    for variable in filter(lambda x: data_type_dict[x]=='nominal',
                           data_type_dict.keys()):
 
        #First we create the columns with dummy variables.
        #Note that the argument 'prefix' means the column names will be
        #prefix_value for each unique value in the original column, so
        #we set the prefix to be the name of the original variable.
        dummy_df=pd.get_dummies(data[variable], prefix=variable)
 
        #Remove old variable from dictionary.
        data_type_dict.pop(variable)
 
        #Add new dummy variables to dictionary.
        for dummy_variable in dummy_df.columns:
            data_type_dict[dummy_variable] = 'nominal'
 
        #Add dummy variables to main df.
        data=data.drop(variable, axis=1)
        data=data.join(dummy_df)
 
    return [data, data_type_dict]

In [53]:
dummied_df = dummy_variables(df3, data_type_dict)[0]

In [58]:
dummied_df.columns.values.tolist()

['_projectid',
 '_teacher_acctid',
 '_schoolid',
 'school_state',
 'school_charter',
 'school_magnet',
 'school_year_round',
 'school_nlns',
 'school_kipp',
 'school_charter_ready_promise',
 'teacher_teach_for_america',
 'teacher_ny_teaching_fellow',
 'vendor_shipping_charges',
 'total_price_excluding_optional_support',
 'total_price_including_optional_support',
 'students_reached',
 'date_posted',
 'RESP',
 'month',
 'quarter',
 'year',
 'time_to_expire',
 'optional_support',
 'school_previous_projects',
 'teacher_previous_projects',
 'log_price_including',
 'log_price_excluding',
 'sqrt_students_reached',
 'price_per_student',
 'total_state_donors',
 'total_state_projects',
 'state_avg_donors',
 'primary_focus_area_Applied Learning',
 'primary_focus_area_Health & Sports',
 'primary_focus_area_History & Civics',
 'primary_focus_area_Literacy & Language',
 'primary_focus_area_Math & Science',
 'primary_focus_area_Music & The Arts',
 'primary_focus_area_Special Needs',
 'primary_focus_s